# 📊 안전 운전 데이터 탐색적 데이터 분석 (EDA)

이 노트북은 SafeDriving 프로젝트의 데이터를 탐색하고 분석하는 과정을 다룹니다.

## 🎯 분석 목표
1. **데이터 이해**: 수집된 데이터의 구조와 특성 파악
2. **패턴 발견**: 안전 운전과 관련된 패턴 및 인사이트 도출
3. **피처 중요도**: 안전 운전 예측에 중요한 변수들 식별
4. **데이터 품질**: 결측값, 이상값, 분포 특성 분석

---

## 📚 라이브러리 및 모듈 import

In [ ]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

# 통계 및 분석
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 시스템 경로 설정
import sys
import os
sys.path.append('../src')

# 프로젝트 모듈
from data.data_loader import SafeDrivingDataLoader
from data.preprocessor import SafeDrivingPreprocessor

# 설정
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

# 시각화 설정
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# 한글 폰트 설정 (한국어 사용시)
plt.rcParams['axes.unicode_minus'] = False

print("📊 라이브러리 import 완료!")

## 📁 데이터 로드 및 기본 정보

In [ ]:
# 데이터 로더 초기화
loader = SafeDrivingDataLoader()

# 모든 데이터셋 로드
print("🔄 데이터 로드 중...")
all_data = loader.load_all_data()

print(f"✅ 총 {len(all_data)}개의 데이터셋 로드 완료!")
for name, df in all_data.items():
    print(f"  📋 {name}: {df.shape}")

### 📊 데이터셋 세부 정보

In [ ]:
# 각 데이터셋 상세 정보 출력
for data_name, df in all_data.items():
    print(f"\n{'='*50}")
    print(f"📊 {data_name.upper()} 데이터셋 분석")
    print(f"{'='*50}")
    
    info = loader.get_data_info(df, data_name)
    
    print(f"🔍 기본 정보:")
    print(f"  - Shape: {info['shape']}")
    print(f"  - 수치형 컬럼: {len(info['numeric_columns'])}개")
    print(f"  - 범주형 컬럼: {len(info['categorical_columns'])}개")
    print(f"  - 메모리 사용량: {info['memory_usage'] / 1024 / 1024:.2f} MB")
    
    # 결측값 정보
    missing_count = sum(info['missing_values'].values())
    if missing_count > 0:
        print(f"⚠️ 결측값: {missing_count}개")
        missing_cols = {k: v for k, v in info['missing_values'].items() if v > 0}
        for col, count in missing_cols.items():
            print(f"    - {col}: {count}개 ({count/info['shape'][0]*100:.1f}%)")
    else:
        print(f"✅ 결측값: 없음")
    
    # 타겟 분포 (있는 경우)
    if 'target_distribution' in info:
        print(f"🎯 타겟 분포: {info['target_distribution']}")
        target_ratio = info['target_distribution'].get(1, 0) / info['shape'][0]
        print(f"  - 긍정 클래스 비율: {target_ratio:.2%}")
    
    # 컬럼 미리보기
    print(f"\n📋 컬럼 목록 (처음 10개):")
    for i, col in enumerate(info['columns'][:10]):
        dtype = info['dtypes'][col]
        print(f"  {i+1:2d}. {col:<20} ({dtype})")
    
    if len(info['columns']) > 10:
        print(f"     ... 외 {len(info['columns'])-10}개 컬럼")

## 🔍 Kaggle 데이터 심화 분석

Kaggle Safe Driving 데이터의 특성을 자세히 살펴봅시다.

In [ ]:
# Kaggle 훈련 데이터 선택
if 'kaggle_train' in all_data:
    kaggle_data = all_data['kaggle_train']
    print(f"📊 Kaggle 훈련 데이터 분석: {kaggle_data.shape}")
    print(f"📋 처음 5행 미리보기:")
    display(kaggle_data.head())
else:
    print("⚠️ Kaggle 훈련 데이터를 찾을 수 없습니다.")

### 🎯 타겟 변수 분석

In [ ]:
if 'kaggle_train' in all_data and 'target' in kaggle_data.columns:
    # 타겟 분포 시각화
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # 막대 그래프
    target_counts = kaggle_data['target'].value_counts()
    axes[0].bar(target_counts.index, target_counts.values, color=['skyblue', 'salmon'])
    axes[0].set_title('타겟 변수 분포', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('타겟 값')
    axes[0].set_ylabel('빈도')
    
    # 비율 표시
    for i, v in enumerate(target_counts.values):
        axes[0].text(i, v + target_counts.max() * 0.01, 
                    f'{v:,}\n({v/len(kaggle_data)*100:.1f}%)', 
                    ha='center', va='bottom', fontweight='bold')
    
    # 파이 차트
    labels = ['안전 (0)', '위험 (1)']
    colors = ['lightblue', 'lightcoral']
    axes[1].pie(target_counts.values, labels=labels, colors=colors, autopct='%1.1f%%',
               startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
    axes[1].set_title('타겟 변수 비율', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # 클래스 불균형 분석
    imbalance_ratio = target_counts[0] / target_counts[1]
    print(f"\n📈 클래스 불균형 분석:")
    print(f"  - 안전(0): {target_counts[0]:,}개 ({target_counts[0]/len(kaggle_data)*100:.1f}%)")
    print(f"  - 위험(1): {target_counts[1]:,}개 ({target_counts[1]/len(kaggle_data)*100:.1f}%)")
    print(f"  - 불균형 비율: {imbalance_ratio:.1f}:1")
    
    if imbalance_ratio > 10:
        print(f"⚠️ 심한 클래스 불균형이 감지되었습니다. SMOTE나 클래스 가중치 조정을 고려해야 합니다.")
    elif imbalance_ratio > 5:
        print(f"⚠️ 중간 정도의 클래스 불균형이 있습니다. 균형 조정 기법을 고려해볼 수 있습니다.")
    else:
        print(f"✅ 클래스 균형이 비교적 양호합니다.")
else:
    print("⚠️ 타겟 변수를 찾을 수 없습니다.")

### 🔢 수치형 변수 분석

In [ ]:
if 'kaggle_train' in all_data:
    # 수치형 컬럼 선별
    numeric_cols = kaggle_data.select_dtypes(include=[np.number]).columns.tolist()
    if 'id' in numeric_cols:
        numeric_cols.remove('id')
    if 'target' in numeric_cols:
        numeric_cols.remove('target')
    
    print(f"📊 수치형 변수 {len(numeric_cols)}개 분석")
    
    # 기본 통계량
    print(f"\n📈 기본 통계량:")
    display(kaggle_data[numeric_cols].describe().round(3))
    
    # 상관관계 분석 (샘플링으로 속도 향상)
    sample_size = min(len(kaggle_data), 5000)
    sample_data = kaggle_data.sample(n=sample_size, random_state=42)
    
    # 타겟과의 상관관계가 높은 변수들 찾기
    if 'target' in kaggle_data.columns:
        correlations = sample_data[numeric_cols + ['target']].corr()['target'].drop('target')
        correlations_abs = correlations.abs().sort_values(ascending=False)
        
        print(f"\n🎯 타겟과 상관관계가 높은 변수 TOP 10:")
        for i, (var, corr) in enumerate(correlations_abs.head(10).items()):
            print(f"  {i+1:2d}. {var:<20} : {correlations[var]:>7.4f} (|{corr:.4f}|)")
        
        # 상관관계 히트맵 (상위 변수들만)
        top_vars = correlations_abs.head(15).index.tolist() + ['target']
        
        plt.figure(figsize=(12, 10))
        corr_matrix = sample_data[top_vars].corr()
        
        # 마스크 생성 (상삼각형 숨김)
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        
        sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='RdYlBu_r', center=0,
                   square=True, fmt='.3f', cbar_kws={"shrink": .8})
        plt.title('주요 변수들의 상관관계 히트맵', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
    
else:
    print("⚠️ Kaggle 데이터를 찾을 수 없습니다.")

### 📊 주요 변수 분포 분석

In [ ]:
if 'kaggle_train' in all_data and len(numeric_cols) > 0:
    # 상위 중요 변수들의 분포 시각화
    if 'target' in kaggle_data.columns:
        top_vars = correlations_abs.head(8).index.tolist()
    else:
        top_vars = numeric_cols[:8]
    
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.ravel()
    
    for i, var in enumerate(top_vars):
        # 히스토그램
        axes[i].hist(kaggle_data[var].dropna(), bins=50, alpha=0.7, edgecolor='black')
        axes[i].set_title(f'{var}', fontweight='bold')
        axes[i].set_ylabel('빈도')
        
        # 기본 통계량 표시
        mean_val = kaggle_data[var].mean()
        median_val = kaggle_data[var].median()
        axes[i].axvline(mean_val, color='red', linestyle='--', alpha=0.8, label=f'평균: {mean_val:.3f}')
        axes[i].axvline(median_val, color='green', linestyle='--', alpha=0.8, label=f'중앙값: {median_val:.3f}')
        axes[i].legend(fontsize=8)
        
        # 이상값 표시
        Q1 = kaggle_data[var].quantile(0.25)
        Q3 = kaggle_data[var].quantile(0.75)
        IQR = Q3 - Q1
        outlier_threshold = Q3 + 1.5 * IQR
        outliers = kaggle_data[var] > outlier_threshold
        
        if outliers.sum() > 0:
            axes[i].axvline(outlier_threshold, color='orange', linestyle=':', alpha=0.8)
            axes[i].text(0.95, 0.95, f'이상값: {outliers.sum()}개', 
                        transform=axes[i].transAxes, ha='right', va='top',
                        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.suptitle('주요 변수들의 분포', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # 분포 특성 분석
    print(f"\n📊 분포 특성 분석:")
    for var in top_vars:
        data = kaggle_data[var].dropna()
        
        # 왜도와 첨도
        skewness = stats.skew(data)
        kurtosis = stats.kurtosis(data)
        
        # 정규성 검정 (샘플링)
        if len(data) > 5000:
            test_sample = data.sample(5000, random_state=42)
        else:
            test_sample = data
        
        _, p_value = stats.normaltest(test_sample)
        
        print(f"  📈 {var:<20}: 왜도={skewness:>6.2f}, 첨도={kurtosis:>6.2f}, p-value={p_value:.2e}")
        
        if abs(skewness) > 2:
            print(f"      ⚠️ 심한 비대칭 분포 (로그 변환 고려)")
        elif abs(skewness) > 1:
            print(f"      ⚠️ 중간 정도 비대칭 분포")
        
        if p_value < 0.001:
            print(f"      ⚠️ 비정규분포 (변환 필요)")

else:
    print("⚠️ 분석할 수치형 변수가 없습니다.")

## 🚗 센서 데이터 분석

GPS, 가속도계, 자이로스코프 등 센서 데이터의 특성을 분석합니다.

In [ ]:
# 센서 데이터 선별
sensor_datasets = {k: v for k, v in all_data.items() if 'sensor' in k}

print(f"🔍 센서 데이터셋 {len(sensor_datasets)}개 발견:")
for name, df in sensor_datasets.items():
    print(f"  📡 {name}: {df.shape}")

# 센서 데이터 통합 분석
if sensor_datasets:
    # 첫 번째 센서 데이터셋으로 상세 분석
    first_sensor = list(sensor_datasets.keys())[0]
    sensor_data = sensor_datasets[first_sensor]
    
    print(f"\n📊 {first_sensor} 데이터 상세 분석:")
    print(f"📋 컬럼 정보:")
    display(sensor_data.info())
    print(f"\n📋 처음 5행:")
    display(sensor_data.head())
    
else:
    print("⚠️ 센서 데이터를 찾을 수 없습니다.")

### 🗺️ GPS 데이터 분석 (있는 경우)

In [ ]:
# GPS 데이터 찾기
gps_data = None
for name, df in sensor_datasets.items():
    if 'gps' in name and all(col in df.columns for col in ['latitude', 'longitude']):
        gps_data = df
        print(f"🗺️ GPS 데이터 발견: {name}")
        break

if gps_data is not None:
    # GPS 데이터 기본 통계
    print(f"\n📊 GPS 데이터 기본 통계:")
    gps_stats = gps_data[['latitude', 'longitude']].describe()
    display(gps_stats)
    
    # 이동 범위 계산
    lat_range = gps_data['latitude'].max() - gps_data['latitude'].min()
    lon_range = gps_data['longitude'].max() - gps_data['longitude'].min()
    
    print(f"\n🗺️ 이동 범위:")
    print(f"  - 위도 범위: {lat_range:.6f}° ({lat_range * 111:.1f} km)")
    print(f"  - 경도 범위: {lon_range:.6f}° ({lon_range * 111:.1f} km)")
    
    # GPS 궤적 시각화
    plt.figure(figsize=(12, 8))
    
    # 전체 궤적
    plt.scatter(gps_data['longitude'], gps_data['latitude'], 
               c=range(len(gps_data)), cmap='viridis', alpha=0.6, s=1)
    plt.colorbar(label='시간 순서')
    plt.xlabel('경도 (Longitude)')
    plt.ylabel('위도 (Latitude)')
    plt.title('GPS 이동 궤적', fontsize=14, fontweight='bold')
    
    # 시작점과 끝점 표시
    plt.scatter(gps_data['longitude'].iloc[0], gps_data['latitude'].iloc[0], 
               color='green', s=100, marker='o', label='시작점', zorder=5)
    plt.scatter(gps_data['longitude'].iloc[-1], gps_data['latitude'].iloc[-1], 
               color='red', s=100, marker='s', label='끝점', zorder=5)
    
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 속도 분석 (있는 경우)
    if 'speed_kmh' in gps_data.columns:
        plt.figure(figsize=(15, 5))
        
        # 시간별 속도 변화
        plt.subplot(1, 2, 1)
        plt.plot(gps_data.index, gps_data['speed_kmh'], linewidth=0.8, alpha=0.7)
        plt.title('시간별 속도 변화', fontweight='bold')
        plt.xlabel('시간 인덱스')
        plt.ylabel('속도 (km/h)')
        plt.grid(True, alpha=0.3)
        
        # 속도 분포
        plt.subplot(1, 2, 2)
        plt.hist(gps_data['speed_kmh'].dropna(), bins=50, alpha=0.7, edgecolor='black')
        plt.axvline(gps_data['speed_kmh'].mean(), color='red', linestyle='--', 
                   label=f'평균: {gps_data["speed_kmh"].mean():.1f} km/h')
        plt.title('속도 분포', fontweight='bold')
        plt.xlabel('속도 (km/h)')
        plt.ylabel('빈도')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # 속도 통계
        print(f"\n🚗 속도 통계:")
        print(f"  - 평균 속도: {gps_data['speed_kmh'].mean():.1f} km/h")
        print(f"  - 최대 속도: {gps_data['speed_kmh'].max():.1f} km/h")
        print(f"  - 속도 표준편차: {gps_data['speed_kmh'].std():.1f} km/h")
        
        # 위험 속도 구간 분석
        high_speed = (gps_data['speed_kmh'] > 80).sum()
        very_high_speed = (gps_data['speed_kmh'] > 120).sum()
        
        print(f"  - 고속 구간 (>80km/h): {high_speed}회 ({high_speed/len(gps_data)*100:.1f}%)")
        print(f"  - 초고속 구간 (>120km/h): {very_high_speed}회 ({very_high_speed/len(gps_data)*100:.1f}%)")

else:
    print("⚠️ GPS 데이터를 찾을 수 없습니다.")

### 📱 가속도계 데이터 분석

In [ ]:
# 가속도계 데이터 찾기
acc_data = None
for name, df in sensor_datasets.items():
    if 'accelerometer' in name or any('acc_' in col for col in df.columns):
        acc_data = df
        print(f"📱 가속도계 데이터 발견: {name}")
        break

if acc_data is not None:
    # 가속도 컬럼 식별
    acc_cols = [col for col in acc_data.columns if col.startswith('acc_')]
    
    if len(acc_cols) >= 3:
        print(f"\n📊 가속도계 데이터 분석 ({len(acc_cols)}개 축):")
        
        # 가속도 기본 통계
        acc_stats = acc_data[acc_cols].describe()
        display(acc_stats)
        
        # 3축 가속도 시각화
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # 시계열 플롯
        sample_size = min(1000, len(acc_data))
        sample_indices = np.linspace(0, len(acc_data)-1, sample_size, dtype=int)
        
        axes[0, 0].plot(sample_indices, acc_data[acc_cols[0]].iloc[sample_indices], label=acc_cols[0], alpha=0.7)
        axes[0, 0].plot(sample_indices, acc_data[acc_cols[1]].iloc[sample_indices], label=acc_cols[1], alpha=0.7)
        axes[0, 0].plot(sample_indices, acc_data[acc_cols[2]].iloc[sample_indices], label=acc_cols[2], alpha=0.7)
        axes[0, 0].set_title('3축 가속도 시계열', fontweight='bold')
        axes[0, 0].set_xlabel('시간 인덱스')
        axes[0, 0].set_ylabel('가속도 (m/s²)')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # 총 가속도 크기 계산
        total_acc = np.sqrt(acc_data[acc_cols[0]]**2 + acc_data[acc_cols[1]]**2 + acc_data[acc_cols[2]]**2)
        
        # 총 가속도 분포
        axes[0, 1].hist(total_acc.dropna(), bins=50, alpha=0.7, edgecolor='black')
        axes[0, 1].axvline(total_acc.mean(), color='red', linestyle='--', 
                          label=f'평균: {total_acc.mean():.2f}')
        axes[0, 1].set_title('총 가속도 크기 분포', fontweight='bold')
        axes[0, 1].set_xlabel('총 가속도 (m/s²)')
        axes[0, 1].set_ylabel('빈도')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # 가속도 변동성 (Jerk) 분석
        jerk = total_acc.diff().abs()
        axes[1, 0].plot(sample_indices[1:], jerk.iloc[sample_indices[1:]], alpha=0.7)
        axes[1, 0].set_title('가속도 변동성 (Jerk)', fontweight='bold')
        axes[1, 0].set_xlabel('시간 인덱스')
        axes[1, 0].set_ylabel('Jerk (m/s³)')
        axes[1, 0].grid(True, alpha=0.3)
        
        # 고가속도 이벤트 탐지
        high_acc_threshold = total_acc.quantile(0.95)
        high_acc_events = total_acc > high_acc_threshold
        
        axes[1, 1].scatter(range(len(total_acc)), total_acc, alpha=0.3, s=1)
        axes[1, 1].scatter(range(len(total_acc))[high_acc_events], 
                          total_acc[high_acc_events], color='red', s=2, 
                          label=f'고가속도 이벤트 ({high_acc_events.sum()}개)')
        axes[1, 1].axhline(high_acc_threshold, color='red', linestyle='--', alpha=0.7)
        axes[1, 1].set_title('고가속도 이벤트 탐지', fontweight='bold')
        axes[1, 1].set_xlabel('시간 인덱스')
        axes[1, 1].set_ylabel('총 가속도 (m/s²)')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # 가속도 통계 요약
        print(f"\n📊 가속도 분석 결과:")
        print(f"  - 평균 총 가속도: {total_acc.mean():.2f} ± {total_acc.std():.2f} m/s²")
        print(f"  - 최대 총 가속도: {total_acc.max():.2f} m/s²")
        print(f"  - 고가속도 이벤트 (상위 5%): {high_acc_events.sum()}개 ({high_acc_events.sum()/len(total_acc)*100:.1f}%)")
        print(f"  - 평균 Jerk: {jerk.mean():.2f} ± {jerk.std():.2f} m/s³")
        
        # 중력 제거 분석 (Z축이 중력 방향이라고 가정)
        if 'acc_z' in acc_cols:
            gravity_removed = np.sqrt(acc_data['acc_x']**2 + acc_data['acc_y']**2 + (acc_data['acc_z'] - 9.8)**2)
            print(f"  - 중력 제거 후 평균 가속도: {gravity_removed.mean():.2f} ± {gravity_removed.std():.2f} m/s²")
    
    else:
        print(f"⚠️ 3축 가속도 데이터가 완전하지 않습니다. 발견된 컬럼: {acc_cols}")

else:
    print("⚠️ 가속도계 데이터를 찾을 수 없습니다.")

## 🧠 피처 엔지니어링 미리보기

전처리 모듈을 사용해서 파생 변수들을 생성하고 그 효과를 살펴봅시다.

In [ ]:
# 전처리 모듈 초기화
preprocessor = SafeDrivingPreprocessor()

# 센서 데이터로 피처 엔지니어링 테스트
if sensor_datasets:
    # 첫 번째 센서 데이터 선택
    test_data = list(sensor_datasets.values())[0].copy()
    
    print(f"🔧 피처 엔지니어링 테스트")
    print(f"원본 데이터: {test_data.shape}")
    
    # 파생 변수 생성
    enhanced_data = preprocessor.create_derived_features(test_data)
    print(f"파생 변수 추가 후: {enhanced_data.shape}")
    
    # 새로 생성된 변수들
    new_features = [col for col in enhanced_data.columns if col not in test_data.columns]
    print(f"\n🆕 새로 생성된 피처 {len(new_features)}개:")
    
    for i, feature in enumerate(new_features[:15]):  # 상위 15개만 표시
        print(f"  {i+1:2d}. {feature}")
    
    if len(new_features) > 15:
        print(f"     ... 외 {len(new_features)-15}개")
    
    # 중요한 안전 운전 피처들 시각화
    safety_features = [col for col in new_features 
                      if any(keyword in col.lower() for keyword in 
                            ['harsh', 'speed_change', 'acceleration', 'turn'])]
    
    if len(safety_features) > 0:
        n_features = min(4, len(safety_features))
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        axes = axes.ravel()
        
        for i in range(n_features):
            feature = safety_features[i]
            data = enhanced_data[feature].dropna()
            
            if len(data) > 0:
                axes[i].hist(data, bins=30, alpha=0.7, edgecolor='black')
                axes[i].set_title(feature, fontweight='bold')
                axes[i].set_ylabel('빈도')
                axes[i].grid(True, alpha=0.3)
                
                # 기본 통계량 표시
                mean_val = data.mean()
                axes[i].axvline(mean_val, color='red', linestyle='--', 
                               label=f'평균: {mean_val:.3f}')
                axes[i].legend()
        
        plt.suptitle('주요 안전 운전 파생 피처들', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # 안전 운전 점수 피처 생성 테스트
        scored_data = preprocessor.create_driving_score_features(enhanced_data)
        
        score_features = [col for col in scored_data.columns 
                         if 'score' in col.lower() or 'freq' in col.lower()]
        
        if score_features:
            print(f"\n🏆 안전 운전 점수 피처들:")
            for feature in score_features:
                data = scored_data[feature].dropna()
                if len(data) > 0:
                    print(f"  - {feature:<25}: 평균 {data.mean():.3f} (범위: {data.min():.3f} ~ {data.max():.3f})")

else:
    print("⚠️ 피처 엔지니어링 테스트를 위한 센서 데이터가 없습니다.")

## 📈 데이터 품질 종합 평가

In [ ]:
print("📊 데이터 품질 종합 평가 리포트")
print("=" * 50)

total_datasets = len(all_data)
total_samples = sum(df.shape[0] for df in all_data.values())
total_features = sum(df.shape[1] for df in all_data.values())

print(f"\n📋 전체 데이터 요약:")
print(f"  - 데이터셋 수: {total_datasets}개")
print(f"  - 총 샘플 수: {total_samples:,}개")
print(f"  - 총 피처 수: {total_features:,}개")

# 데이터셋별 품질 평가
quality_scores = []

for name, df in all_data.items():
    # 품질 지표 계산
    missing_ratio = df.isnull().sum().sum() / (df.shape[0] * df.shape[1])
    duplicate_ratio = df.duplicated().sum() / df.shape[0]
    
    # 수치형 변수의 분산
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        variance_score = df[numeric_cols].var().mean()
    else:
        variance_score = 0
    
    # 종합 품질 점수 (0-100)
    quality_score = (
        (1 - missing_ratio) * 40 +  # 결측값이 적을수록 좋음
        (1 - duplicate_ratio) * 30 +  # 중복이 적을수록 좋음
        min(variance_score / 10, 1) * 30  # 적당한 분산이 좋음
    ) * 100
    
    quality_scores.append({
        'dataset': name,
        'samples': df.shape[0],
        'features': df.shape[1],
        'missing_ratio': missing_ratio,
        'duplicate_ratio': duplicate_ratio,
        'quality_score': quality_score
    })

# 품질 점수 데이터프레임
quality_df = pd.DataFrame(quality_scores)
quality_df = quality_df.sort_values('quality_score', ascending=False)

print(f"\n🏆 데이터셋별 품질 점수 (100점 만점):")
print("-" * 80)
for _, row in quality_df.iterrows():
    score_emoji = "🟢" if row['quality_score'] >= 80 else "🟡" if row['quality_score'] >= 60 else "🔴"
    print(f"{score_emoji} {row['dataset']:<20}: {row['quality_score']:>5.1f}점 "
          f"(결측률: {row['missing_ratio']*100:>4.1f}%, 중복률: {row['duplicate_ratio']*100:>4.1f}%)")

# 품질 개선 제안
print(f"\n💡 품질 개선 제안:")

low_quality = quality_df[quality_df['quality_score'] < 70]
if len(low_quality) > 0:
    print(f"  ⚠️ 품질이 낮은 데이터셋 {len(low_quality)}개:")
    for _, row in low_quality.iterrows():
        print(f"    - {row['dataset']}: ", end="")
        if row['missing_ratio'] > 0.1:
            print("결측값 처리 필요 ", end="")
        if row['duplicate_ratio'] > 0.05:
            print("중복값 제거 필요 ", end="")
        print()
else:
    print(f"  ✅ 모든 데이터셋이 양호한 품질을 보입니다!")

# 전체 품질 점수
overall_quality = quality_df['quality_score'].mean()
overall_emoji = "🟢" if overall_quality >= 80 else "🟡" if overall_quality >= 60 else "🔴"

print(f"\n{overall_emoji} 전체 데이터 품질 점수: {overall_quality:.1f}점")

if overall_quality >= 80:
    print(f"  ✅ 우수한 데이터 품질! 모델 학습에 적합합니다.")
elif overall_quality >= 60:
    print(f"  ⚠️ 보통 수준의 데이터 품질. 일부 전처리가 필요합니다.")
else:
    print(f"  🔴 데이터 품질 개선이 시급합니다. 상당한 전처리 작업이 필요합니다.")

## 🎯 주요 발견사항 및 다음 단계

### 📊 주요 발견사항

이 EDA를 통해 발견된 주요 인사이트들을 정리합니다:

1. **데이터 구조 분석**
   - 전체 데이터셋 구성과 각각의 특성 파악 완료
   - 결측값과 이상값 패턴 식별

2. **안전 운전 패턴**
   - 타겟 변수의 클래스 불균형 정도 확인
   - 주요 위험 요인들과의 상관관계 분석

3. **센서 데이터 특성**
   - GPS 궤적과 속도 패턴 분석
   - 가속도계 데이터의 노이즈와 신호 특성

4. **피처 엔지니어링 가능성**
   - 급정지/급가속 감지 가능성 확인
   - 운전 패턴 기반 안전 점수 계산 가능성

### 🔄 다음 단계 (Phase 3)

1. **피처 엔지니어링 심화**
   - 운전 패턴 기반 피처 개발
   - 시간 윈도우 기반 집계 피처 생성

2. **모델 개발 준비**
   - 클래스 불균형 해결 전략 수립
   - 피처 선택 및 차원 축소 검토

3. **베이스라인 모델 구축**
   - XGBoost/LightGBM 모델 개발
   - PyTorch MLP 모델 구축

---

**다음 노트북**: `02_Feature_Engineering.ipynb` 에서 더 상세한 피처 엔지니어링을 진행합니다.

In [ ]:
print("🎉 EDA 완료!")
print("다음 단계: 피처 엔지니어링 및 모델 개발")

# 분석 결과 요약 저장 (선택사항)
import json
from datetime import datetime

eda_summary = {
    'analysis_date': datetime.now().isoformat(),
    'total_datasets': len(all_data),
    'total_samples': sum(df.shape[0] for df in all_data.values()),
    'total_features': sum(df.shape[1] for df in all_data.values()),
    'data_quality': {
        'overall_score': overall_quality,
        'datasets': quality_scores
    },
    'next_steps': [
        'Feature Engineering',
        'Model Development',
        'Hyperparameter Optimization'
    ]
}

# 결과 저장
with open('../results/eda_summary.json', 'w', encoding='utf-8') as f:
    json.dump(eda_summary, f, indent=2, ensure_ascii=False)

print("📄 EDA 요약 결과가 '../results/eda_summary.json'에 저장되었습니다.")